In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from joblib import Parallel, delayed
from numba import njit
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')
dados_concatenados = pd.read_csv('/content/drive/MyDrive/concatenado.csv')

# Caminho para salvar o arquivo no Google Drive
caminho_arquivo_saida = '/content/drive/My Drive/resultados_comparacao.csv'

# Guardar as colunas que interessam para a comparação (excluindo metadados como frame, face_id, etc.)
colunas_comparacao = [coluna for coluna in dados_concatenados.columns if not coluna.startswith(('frame', 'face_id', 'timestamp', 'confidence', 'success', 'arquivo'))]

# Extrair os vetores de interesse para a comparação
matriz_comparacao = dados_concatenados[colunas_comparacao].values

# Calcular a matriz de distâncias L2 (euclideanas) entre os vetores usando Numba
@njit
def calcular_distancias_numba(matriz):
    n = matriz.shape[0]
    distancias = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            distancias[i, j] = np.linalg.norm(matriz[i] - matriz[j])
            distancias[j, i] = distancias[i, j]
    return distancias

# Calcular as distâncias usando a função acelerada pelo Numba
matriz_distancias = calcular_distancias_numba(matriz_comparacao)

# Função para processar um par de índices e calcular a distância
def calcular_distancia(i, j):
    return {
        'arquivo_1': dados_concatenados.iloc[i]['arquivo'],
        'arquivo_2': dados_concatenados.iloc[j]['arquivo'],
        'distancia': matriz_distancias[i, j]
    }

# Paralelizar a execução das comparações
resultados = Parallel(n_jobs=-1)(
    delayed(calcular_distancia)(i, j) for i in range(len(matriz_distancias)) for j in range(i + 1, len(matriz_distancias))
)

# Converter os resultados para um DataFrame
df_resultados = pd.DataFrame(resultados)

# Salvar os resultados em um arquivo CSV no Google Drive
df_resultados.to_csv(caminho_arquivo_saida, index=False)

print("Comparações realizadas e resultados salvos com sucesso.")

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from joblib import Parallel, delayed
import os
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')
dados_concatenados = pd.read_csv('/content/drive/MyDrive/concatenado.csv')

# Caminho para salvar o arquivo no Google Drive
caminho_arquivo_saida = '/content/drive/My Drive/resultados_comparacao.csv'
caminho_arquivo_progresso = '/content/drive/My Drive/progresso_comparacao.csv'

# Guardar as colunas que interessam para a comparação (excluindo metadados como frame, face_id, etc.)
colunas_comparacao = [coluna for coluna in dados_concatenados.columns if not coluna.startswith(('frame', 'face_id', 'timestamp', 'confidence', 'success', 'arquivo'))]

# Extrair os vetores de interesse para a comparação
matriz_comparacao = dados_concatenados[colunas_comparacao].values

# Calcular a matriz de distâncias L2 (euclideanas) entre os vetores
distancias = pdist(matriz_comparacao, metric='euclidean')
matriz_distancias = squareform(distancias)

# Função para carregar o progresso salvo
def carregar_progresso(caminho):
    if os.path.exists(caminho):
        df = pd.read_csv(caminho)
        return df
    else:
        return pd.DataFrame(columns=['arquivo_1', 'arquivo_2', 'distancia'])

# Função para salvar o progresso atual
def salvar_progresso(df, caminho):
    df.to_csv(caminho, index=False)

# Carregar o progresso anterior, se existir
df_resultados = carregar_progresso(caminho_arquivo_progresso)

# Conjunto de pares já processados
pares_processados = set((row['arquivo_1'], row['arquivo_2']) for _, row in df_resultados.iterrows())

# Função para processar um par de índices e calcular a distância
def calcular_distancia(i, j):
    arquivo_1 = dados_concatenados.iloc[i]['arquivo']
    arquivo_2 = dados_concatenados.iloc[j]['arquivo']
    if (arquivo_1, arquivo_2) not in pares_processados and (arquivo_2, arquivo_1) not in pares_processados:
        distancia = matriz_distancias[i, j]
        resultado = {
            'arquivo_1': arquivo_1,
            'arquivo_2': arquivo_2,
            'distancia': distancia
        }
        df_resultados.loc[len(df_resultados)] = resultado
        pares_processados.add((arquivo_1, arquivo_2))
        pares_processados.add((arquivo_2, arquivo_1))
        return resultado
    return None

# Iterar sobre a matriz de distâncias e salvar o progresso periodicamente
try:
    for i in range(len(matriz_distancias)):
        for j in range(i + 1, len(matriz_distancias)):
            resultado = calcular_distancia(i, j)
            if resultado:
                # Salvar o progresso a cada 1000 comparações
                if len(df_resultados) % 1000 == 0:
                    salvar_progresso(df_resultados, caminho_arquivo_progresso)
except KeyboardInterrupt:
    print("Processo interrompido pelo usuário. Salvando progresso...")
    salvar_progresso(df_resultados, caminho_arquivo_progresso)
except Exception as e:
    print(f"Ocorreu um erro: {e}. Salvando progresso...")
    salvar_progresso(df_resultados, caminho_arquivo_progresso)
    raise

# Salvar o progresso final
salvar_progresso(df_resultados, caminho_arquivo_progresso)

# Salvar os resultados finais em um arquivo CSV
df_resultados.to_csv(caminho_arquivo_saida, index=False)

print("Comparações realizadas e resultados salvos com sucesso.")